# Scraper

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scraper.guardianukscraper.spiders.guardian_spider import GuardianSpider

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

def finished():
    print 'I finished'
    
process.crawl(GuardianSpider)
process.start()

finished()

INFO:scrapy.utils.log:Scrapy 1.0.5 started (bot: scrapybot)
2016-03-12 16:23:19 [scrapy] INFO: Scrapy 1.0.5 started (bot: scrapybot)
INFO:scrapy.utils.log:Optional features available: ssl, http11, boto
2016-03-12 16:23:19 [scrapy] INFO: Optional features available: ssl, http11, boto
INFO:scrapy.utils.log:Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2016-03-12 16:23:19 [scrapy] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
INFO:scrapy.middleware:Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
2016-03-12 16:23:19 [scrapy] INFO: Enabled extensions: CloseSpider, TelnetConsole, LogStats, CoreStats, SpiderState
DEBUG:boto:Retrieving credentials from metadata server.
2016-03-12 16:23:19 [boto] DEBUG: Retrieving credentials from metadata server.
ERROR:boto:Caught exception reading instance data
Traceback (most recent call last):
  File "/Users/betoesquivel/Envs/ML/lib

I finished


In [ ]:
import pandas as pd
frame = pd.DataFrame(dir(process)).transpose()


print process.join()
frame